In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
company = pd.read_excel('компании.xlsx', encoding="windows-1251")
products = pd.read_excel('продукты.xlsx', encoding="windows-1251")
acc = pd.read_excel('счета.xlsx', encoding="windows-1251")
tarif = pd.read_excel('тарифы.xlsx', encoding="windows-1251")

Data Preproccesing

In [3]:
acc = acc.rename(columns={'ID_CLIENT': 'ID_Client'})
company = company.rename(columns={'ID_Client': 'ID_Client'})

In [4]:
df_tarif_acc = pd.merge(tarif, acc, on='ID_Client')

In [5]:
df_tarif_acc['Закрыт'] = df_tarif_acc['Закрыт'].replace(to_replace=['Y', 'N'], value=[1, 0])

In [6]:
df_tarif_acc['Статус'] = df_tarif_acc['Статус'].replace(to_replace=['Работает', 'Закрыт'], value=[1, 0])

In [7]:
df_tarif_acc['Филиал'] = df_tarif_acc['Филиал'].map(lambda x: str(x)[18:])

In [8]:
df_1 = pd.merge(df_tarif_acc, company, on='ID_Client')

In [9]:
df_1['TypeClient'] = df_1['TypeClient'].replace(to_replace=['ИП', 'ЮЛ'], value=[1, 2])

In [24]:
products = products.assign(pr1=0)
products = products.assign(pr2=0)
products = products.assign(pr3=0)
products = products.assign(pr4=0)
products = products.assign(pr5=0)
products = products.assign(pr6=0)
products = products.assign(pr7=0)
products = products.assign(pr8=0)
products = products.assign(pr9=0)
products = products.assign(pr10=0)
products = products.assign(pr11=0)
products = products.assign(pr12=0)
products = products.assign(pr13=0)
products = products.assign(pr14=0)
products = products.assign(pr15=0)
products = products.assign(pr16=0)
products = products.assign(pr17=0)
products = products.assign(pr18=0)
products = products.assign(pr19=0)

In [25]:
products['pr1'] = (products['Продукт']== 1)
products['pr2'] = (products['Продукт']== 2)
products['pr3'] = (products['Продукт']== 3)
products['pr4'] = (products['Продукт']== 4)
products['pr5'] = (products['Продукт']== 5)
products['pr6'] = (products['Продукт']== 6)
products['pr7'] = (products['Продукт']== 7)
products['pr8'] = (products['Продукт']== 8)
products['pr9'] = (products['Продукт']== 9)
products['pr10'] = (products['Продукт']== 10)
products['pr11'] = (products['Продукт']== 11)
products['pr12'] = (products['Продукт']== 12)
products['pr13'] = (products['Продукт']== 13)
products['pr14'] = (products['Продукт']== 14)
products['pr15'] = (products['Продукт']== 15)
products['pr16'] = (products['Продукт']== 16)
products['pr17'] = (products['Продукт']== 17)
products['pr18'] = (products['Продукт']== 19)

In [26]:
products = products.rename(columns={'ID_CLIENT': 'ID_Client'})

In [27]:
df_p = products.drop(['Продукт'], axis=1)

In [28]:
df = pd.merge(df_1, df_p, on=['ID_Client'])

Splitting data into test and train datasets

In [31]:
y = df[['pr1', 'pr2','pr3','pr4','pr5','pr6','pr7','pr8','pr9','pr10','pr11', 'pr12', 'pr13', 'pr14', 'pr15', 'pr16', 'pr17', 'pr18', 'pr19']]
X = df.drop(['pr1', 'pr2','pr3','pr4','pr5','pr6','pr7','pr8','pr9','pr10','pr11', 'pr12', 'pr13', 'pr14', 'pr15', 'pr16', 'pr17', 'pr18', 'pr19'], axis = 1)
labels = ['pr1', 'pr2','pr3','pr4','pr5','pr6','pr7','pr8','pr9','pr10','pr11', 'pr12', 'pr13', 'pr14', 'pr15', 'pr16', 'pr17', 'pr18', 'pr19']

Changing all missing data to 0

In [32]:
X = X.fillna(0)

We have dropped ID_Client and ID_ACC in order to logreg to work

In [33]:
X = X.drop(['ID_Client', 'ID_ACC'], axis = 1)

other one finded mistake

In [34]:
X['Статус'] = X['Статус'].replace(to_replace=['Предварительный ввод', 'Ожидает подтверждения'], value=[2, 3])

In [35]:
X.shape

(451521, 22)

In [36]:
from sklearn.model_selection  import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(
    X, y, test_size=0.33, random_state=5)

Implementing Logreg to dataset

In [42]:
y.head()

,pr1,pr2,pr3,pr4,pr5,pr6,pr7,pr8,pr9,pr10,pr11,pr12,pr13,pr14,pr15,pr16,pr17,pr18,pr19
0,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0
1,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0
2,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0
3,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0
4,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0


In [46]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
clf = LogisticRegression()
y_pred = Y_test
for c in labels:
    clf = LogisticRegression()
    clf.fit(X_train, Y_train[c])
    y_pred[c] = clf.predict_proba(X_test)[:,1]
    score = np.mean(cross_val_score(clf, X_train, Y_train[c], scoring='roc_auc', cv=5))
    print("ROC_AUC score for", c, "=",  score)

/Users/macbook/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


ROC_AUC score for pr1 = 0.8453789918763025


/Users/macbook/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


ROC_AUC score for pr2 = 0.7023895157946095


/Users/macbook/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


...

/Users/macbook/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


ROC_AUC score for pr17 = 0.9999400963103637


/Users/macbook/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


ROC_AUC score for pr18 = 0.8312291917425849


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0